# Lesson 1

In this unit we write a poker program. This is an example of a general process where we start with a vague understanding of the problem and then we refine our understanding so that it has a formal specification, then we specify that into something that is amenable to be coded, and we end up with working code. In a diagram this looks like this:

      1              2                     3
?? -----> Problem -----> Specifications -----> Code
  Understand     Specify                Design


The first thing we need to do is to make an *inventory* of the concepts we will have to deal with.

1. **Hands**: a hand consists of 5 cards. A card has a rank and a suit. The program we are trying to write takes a *list of hands* and returns the *best hand*.
2. In order to select the best hand we need to *rank* hands. Another concept is, therefore, **hand rank**. What are the concepts that make a hand rank? This [Wikipedia page](https://en.wikipedia.org/wiki/List_of_poker_hands) details all the rules of poker, but we are primarily interested in the following three concepts:
   1. **n-kind**: when a hand contains $n$ cards of the same *rank* (not suit).
   2. **straight**: when we have *5 consecutive ranks*.
   3. **flush**: when all 5 cards have the same suit, and the ranks don't matter.

Now we are ready to move to the desigh phase.

## Representing hands

What is a good representation of a hand? Consider these three examples:

- `['JS', 'JD', '2S', '2C', '7H']`
- `[(11, 'S'), (11, 'D'), (2, 'S'), (2, 'C'), (7, 'H')]`
- `set(['JS', 'JD', '2S', '2C', '7H'])`
- `'JS JS 2S 2C 7H'`

Which one(s) are best suited to our problem? Which ones are not? The set representation wouldn't work if we used two decks, otherwise all the above representations will work, if not with the same efficacy. For example, the first and the last one are equivalent, but for the string representation you will need to call `split()`. Overall, the first two are the best ones.

## The `poker()` function

We can start thinking about the `poker()` function. Out of a list of hands, this function should return the highest ranking hand, therefore we can intuitively imagine that a call to `max()` should appear here.

In [ ]:
def poker(hands):
    "Return the best hand: poker([hand, ...]) => hand"
    pass

To make sure we understand how `max()` works, look at the example below. The returned values might be counterintuitive at first. Note the use of `key=abs` (and not `key=abs()`). Look at the documentation for [`list.sort()`](https://docs.python.org/3/library/stdtypes.html#list.sort) and at the [Sorting HOWTO](https://docs.python.org/3/howto/sorting.html#sortinghowto).

In [1]:
max([3, 4, 5, 0]), max([3, 4, -5, 0], key=abs)

(5, -5)

For now we can write the `poker()` function based on a yet-to-define `hand_rank()` function.

In [ ]:
def poker(hands):
    "Return the best hand: poker([hand, .   ..]) => hand"
    return max(hands, key=hand_rank)

def hand_rank(hand):
    return None

We should also have some tests in place so that we can check whether our functions are doing the right thing. We include tests for a single hand (which should correspond to the return value) and for 100 hands.

In [ ]:
def test():
    "Test cases for the functions in poker program"
    sf = "6C 7C 8C 9C TC".split() # => ['6C', '7C', '8C', '9C', 'TC']
    fk = "9D 9H 9S 9C 7D".split() # Four of a kind
    fh = "TD TC TH 7C 7D".split() # Full House
    assert poker([sf, fk, fh]) == sf
    assert poker([fk, fh]) == fk
    assert poker([fh, fh]) == fh
    assert poker([fk]) == fk
    assert poker([sf, fk] + [fh] * 98) == fh


## Hand Rank Attempt

This is our most complicated function. It takes a hand, but what does it return? It should return a value that is comparable by `max()`, but we don't yet know what it should be. Why not a number? We can rank hands from 0 to 8, with 0 for an empty hand and 8 for a Straight Flush (note, we are not considering Royal Flushes). We know it is going to be important to look at ranks, so let's imagine we have written a function `card_ranks()` that returns the ranks of the cards in a hand.

We may start enumerating all the cases, starting from a Straight Flush and moving on. Note that in the definition below we are including several functions we have not yet defined, like `straight()`, `flush()` and `kind(n, ranks)`.

```python
def hand_rank(hand):
    "Return a value indicating the ranking of a hand."
    ranks = card_ranks(hand) # We haven't written this one yet.
    if straight(ranks) and flush(hand):
        return 8
    elif kind(4, ranks):
        return 7
    # etc etc
```

The question is, would such a function work? The answer is, sometimes it would, and sometimes it would not. It would work for comparisons between a full house and a straight, for example, but what if two players had a pair of 10s and of 9s respectively. We need to disambiguate and rank these two pairs.

## Representing Rank

We need a better way to represent rank. There are several possibilities. We may continue to use integers, but bigger ones. For example, if we have two four-of-a-kind hands, with ranks [9, 9, 9, 9, 5] and [3, 3, 3, 3, 2] respectively. In the formulation above they would be both ranked as 7. We may represent them as 70905 and 70302, respectively. The first digit would be the rank, the third the rank of the four cards, and the last digit the rank of the remaining card. The zeros allow for ranks greater than 9.
We could otherwise use real numbers, like 7.0905 and 7.0302. Or we could use tuples like (7, 9, 5) and (7, 3, 2).

All these representations would actually work, and `max()` would do the right thing, but the tuple-based representation is the most convenient, since the other require fairly complicated arithmetic.

### Ordering in Tuples

We want to compare tuples. Tuples are compared element by element, just like strings. This is called **lexigraphic ordering**. Let's look at some examples.

In [5]:
print((7, 9, 5) > (7, 3, 2))
print((7, 9, 5) > (7, 9, 4))
print((7, 9, 5) > (7, 9, 5, 1))

True
True
False


## Wild West Poker

To understand the new ordering mechanism we can look at a few examples.

- Straight Flush, Jack High: `['JC', 'TC', '9C', '8C', '7C']`: (8, 11 )
- Four Aces and a Queen Kicker: `['AS', 'AH', 'AD', 'AC', 'QH']`: (7, 14, 12). 14 is the rank of an Ace.
- Full House, 8s over Kings: `['8S', '8H', '8D', 'KS', 'KC']`: (6, 8, 13)
- Flush 10-8: `['TD', '8D', '7D', '5D', '3D']`: usually the two highest cards are enough to break ties, but this is not always the case. In actuality we may want all the ranks, therefore we return (5, [10, 8, 7, 5, 3]). It is still possible that another player will have the same cards in another suit.
- Straight, Jack High: `['JC', 'TS', '9D', '8C', '7C']` (4, 11).
- Three Sevens: `['7H', '7D', '7C', '5C', '2C']`: (3, 7, [7, 7, 7, 5, 2]).
- Two pairs, Jacks and Threes: `['JD', 'JC', '3S', '3H', 'KH']`: (2, 11, 3, [13, 11, 11, 3, 3]).
- Pair of Twos, Jack High: `['2H', '2S', 'JD', '6H', '3C']`: (1, 2, [11, 6, 3, 2, 2]).
- I've got nothing: `['7C', '5C', '4C', '3C', '2D']`: (0, 7, 5, 4, 3, 2)

In some cases, in the examples above we return a tuple that contains a list. We cannot compare a number with a list. For example, this comparison would return an error: `(2, 7, 6) > (2, 7, [7, 7, 5, 5, 2])`.

**TODO** show if and why comparing hands with different representations does not create issues.

Let's go back to `hand_rank()`. Note that we are using `kind(4, ranks)` both as a logical value (in the `elif` statement) and to return the ranks that appears 4 times and 4 times respectively.

In [ ]:
def hand_rank(hand):
    ranks = card_ranks(hand)
    if straight(ranks) and flush(hand):            # straight flush
        return (8, max(ranks))
    elif kind(4, ranks):                           # 4 of a kind
        return (7, kind(4, ranks), kind(1, ranks))
    elif kind(3, ranks) and kind(2, ranks):        # full house
        return (6, kind(3, ranks), kind(2, ranks))
    elif flush(hand):                              # flush
        return (5, ranks)
    elif straight(ranks):                          # straight
        return (4, max(ranks))
    elif kind(3, ranks):                           # 3 of a kind
        return (3, kind(3, ranks), ranks)
    elif two_pair(ranks):                          # 2 pair
        return (2, two_pair(ranks))
    elif kind(2, ranks):                           # kind
        return (1, kind(2, ranks), ranks)
    else:                                          # high card
        return (0, ranks)

We can now define the `card_ranks()` function. Note how simple is the implementation based on `.index(r)`, where we just put two `-` placeholders at the beginning, so that the ranking is correct. My initial attempt used a dictionary. Note also the use of `.sort(reverse=True)`.

In [ ]:
def card_ranks(hand):
    "Return a list of the ranks, sorted with higher first."
    ranks = ['--23456789TJQKA'.index(r) for r, s in hand]
    return ranks.sort(reverse=True)

Similarly, we can implement the `straight()` and `flush()` functions. Note the arguments: `straigh(ranks)` and `flush(hand)`. The former only need the ranks to decide whether it is a straight or not. The latter needs the suits. Since the ranks are needed by other functions, it makes sense to have a `card_ranks(hand)` function that extracts the ranks once.

In [ ]:
def straight(ranks):
    "Return True if the ordered ranks form a 5-card straight."
    return (max(ranks) - min(ranks) == 4) and (len(set(ranks)) == 5)

def flush(hand):
    "Return True if all the cards have the same suit."
    # Note how neat: this only works because all ranks are 1-character long.
    suits = [s for r, s in hand]
    return len(set(suits)) == 1

For the `kind()` function I initially used a counter, but this is not necessary, and we can just use the `.count()` method for lists.

Note that in the code below we use `return r` that will break the loop at the first occurrence of a `True` condition. This is a frequent pattern: when you expect that multiple elements may satisfy a condition, but you are only interested in the first one, `return` is your friend.

In [ ]:
def kind(n, ranks):
    """Return the first rank that this hand has exactly n of.
    Return None if there is no n-of-a-kind in the hand."""
    for r in ranks:
        if ranks.count(r) == n:
            return r
        else:
            return None

We can also update our `test()` function.

In [10]:
def test():
    "Test cases for the functions in poker program"
    sf = "6C 7C 8C 9C TC".split() # Straight Flush
    fk = "9D 9H 9S 9C 7D".split() # Four of a Kind
    fh = "TD TC TH 7C 7D".split() # Full House
    assert poker([sf, fk, fh]) == sf
    assert poker([fk, fh]) == fk
    assert poker([fh, fh]) == fh
    assert poker([sf]) == sf
    assert poker([sf] + 99*[fh]) == sf
    # New assertions
    assert hand_rank(sf) == (8, 10)
    assert hand_rank(fk) == (7, 9, 7)
    assert hand_rank(fh) == (6, 10, 7)
    assert card_ranks(sf) == [10, 9, 8, 7, 6]
    assert card_ranks(fk) == [9, 9, 9, 9, 7]
    assert card_ranks(fh) == [10, 10, 10, 7, 7]